In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture --no-stderr
!pip install python-dotenv openAI langchain_core langchain_openai

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [ ]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

In [ ]:
llm.invoke("안녕")

'하세요\n\n��는 �����트��어 개��자이고 한 가지 ��문이 있습니다.\n\n개��을 하다 보면 자주 사용하는 코드를 함수로 만들어서 ��사용하고 ��은 경우가 있습니다. 이��� �����게 하면 ��을��요?\n\n��를 ��어, ��번 사용하는 로그인 함수를 만들어서 다른 코드에서 호출하고 ��은 경우입니다. 이�� 경우에는 �����게 코드를 작성해야 할��요?\n\n���� 부���드��니다.\n\n��사합니다.'

In [ ]:
# <오픈AI API를 사용하여 주제에 대한 간단한 설명 생성 파이프라인 구축>

# 라이브러리 불러오기
import openai
from typing import List

# 기본 오픈AI 클라이언트 사용
client = openai.OpenAI()

# "안녀하세요!" 메시지를 보내고 응답을 받음
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "안녕하세요!"}]
)
response.choices[0].message.content

'안녕하세요! 어떻게 도와드릴까요?'

In [ ]:
# 요청에 사용할 프롬프트 템플릿 정의
prompt_template = "주제 {topic}에 대해 짧은 설명을 해주세요"

# 메시지를 보내고 모델의 응답을 받는 함수
def call_chat_model(messages: List[dict]) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    return response.choices[0].message.content

# 주어진 주제에 따라 설명을 요청하는 함수
def invoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

# "더블딥" 주제로 설명 요청
invoke_chain("더블딥")

'"더블딥"은 경제학에서 사용하는 용어로, 경제가 두 번의 침체를 겪고 나서 회복하는 현상을 의미합니다. 첫 번째 침체 후에 일시적인 회복이 나타나지만, 이후에 다시 경제가 하락하는 경우를 말합니다. 이 현상은 일반적으로 경기 순환의 긴축적인 패턴을 나타내며, 경제 정책이나 시장의 신뢰도 등에 따라 발생할 수 있습니다. 더블딥은 투자자와 정책 입안자들에게 중요한 신호로 작용하며, 경제의 전반적인 건강 상태를 평가하는 데 도움을 줍니다.'

In [ ]:
# <랭체인을 사용하여 주제에 대한 간단한 설명 생성 파이프라인 구축>

# 라이브러리 불러오기
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
# 미스트랄AI 모델을 사용할 경우 주석 해제
# from langchain_mistralai.chat_models import ChatMistralAI

# 주어진 주제에 대해 짧은 설명을 요청하는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template(
    "주제 {topic}에 대해 짧은 설명을 해주세요"
)

# 출력 파서를 문자열로 설정
output_parser = StrOutputParser()
# 오픈AI의 gpt-4o 모델을 사용하여 채팅 모델 설정
model = ChatOpenAI(model="gpt-4o")
# 미스트랄AI 모델을 사용할 경우 주석 해제
# model = ChatMistralAI()

# 파이프라인 설정: 주제를 받아 프롬프트를 생성하고, 모델을 통해 응답을 생성한 후 문자열로 파싱
chain = (
    {"topic": RunnablePassthrough()}  # 입력 받은 주제를 그대로 통과시킴
    | prompt                         # 프롬프트 템플릿을 적용
    | model                          # 모델을 사용해 응답 생성
    | output_parser                  # 응답을 문자열로 파싱
)

# "더블딥" 주제로 설명 요청
chain.invoke("더블딥")

'"더블딥(Double Dip)"은 경제 분야에서 주로 사용되는 용어로, 경기 침체가 일단 끝난 뒤 경제가 다시 회복되는 듯하다가 또 다시 침체로 접어드는 현상을 말합니다. 즉, 경제 성장률이 음수로 돌아섰다가 잠시 플러스로 전환된 이후 다시 음수로 떨어지는 두 번의 침체가 연속적으로 발생하는 것을 지칭합니다. 이는 소비 감소, 투자 부진, 외부 충격 등 여러 요인에 의해 발생할 수 있으며, 정책 결정자들에게 도전 과제가 될 수 있습니다.'